In [1]:
import galprime
import sys
import os
from astropy.io import fits
from astropy.table import Table
from numpy.random import normal
import numpy as np

from scipy.interpolate import interp1d

from tqdm import tqdm
from matplotlib import pyplot as plt

import importlib
importlib.reload(galprime)

import dask.bag as db

In [2]:

config = galprime.load_config_file('../newconfig.galprime')
config["CORES"] = 2
# galprime.print_config(config)
# sys.exit()

# Load in backgrounds and psfs
bgs = galprime.cutouts_from_file("../cutouts.fits")
psfs = galprime.cutouts_from_file('../i_psfs.fits')

# TODO change this to the actual COSMOS mag table once I have it available to me

cosmos_table = Table.read("../COSMOS_V9_MAGS.fits", format="fits")

mags = cosmos_table["i"][cosmos_table["i"] > 0]

mag_kde = galprime.gaussian_kde(mags)

gp = galprime.GPrime(config='../newconfig.galprime', backgrounds=bgs, mag_kde=mag_kde, psfs=psfs)


# test_container.save_outputs(filename="GPrime_" + str(test_container.metadata["ID"]) + ".fits")

container_list = gp.pipeline(max_bins=1, debug=True, process_method=galprime.gprime_single, progress_bar=True)



Running GalPRIME, version 2.1.2
-12 bins in total. Running GalPRIME on 1 bins total
-Backgrounds were supplied: 50 in total.
Working through 10 models for bin: [ 9.6 10.   0.1  0.3  0.   0.5] with 2 cores.


Bin: 0_0_0_:  30%|███       | 3/10 [00:09<00:18,  2.68s/it]

Function timed out.

Bin: 0_0_0_:  40%|████      | 4/10 [00:23<00:38,  6.41s/it]


Function timed out.

...

Bin: 0_0_0_:  70%|███████   | 7/10 [00:29<00:10,  3.46s/it]

Function timed out.

Bin: 0_0_0_: 11it [00:43,  3.98s/it]                       


 . Profiles cleaned 7 7


Finished. Time elapsed: 0.758 minutes.


In [ ]:
import dask
from dask.distributed import Client, LocalCluster, Worker
from dask_jobqueue import SLURMCluster

In [ ]:
print(config["TIME_LIMIT"])

15.0


In [ ]:

cluster = LocalCluster(n_workers=config["CORES"], 
                       threads_per_worker=1, 
                       lifetime=str(config["TIME_LIMIT"]) + "s")
client = Client(cluster)



In [ ]:
def load_and_process(filename):
    container = galprime.load_pickle(filename)
    return container.metadata
    
    # try:
    #     container.process_object(plot=False)
    #     print("Completed:", filename)
    # except Exception as error:
    #     print("Failed:", filename, error)
    # return container

A = client.map(load_and_process, container_list)

In [ ]:

cluster = LocalCluster(n_workers=config["CORES"], 
                       threads_per_worker=1, 
                       lifetime=str(config["TIME_LIMIT"]) + "s")
client = Client(cluster)

A = client.map(load_and_process, container_list)

distributed.active_memory_manager - WARNING - Tried retiring worker tcp://127.0.0.1:63329, but 10 tasks could not be moved as there are no suitable workers to receive them. The worker will not be retired.
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOLoop object at 0x10e283400>>, <Task finished name='Task-6' coro=<Worker.handle_scheduler() done, defined at /Users/hsouch/opt/miniconda3/lib/python3.9/site-packages/distributed/worker.py:1401> exception=CommClosedError('in <TCP (closed) ConnectionPool.close_gracefully local=tcp://127.0.0.1:65478 remote=tcp://127.0.0.1:63247>: Stream is closed')>)
Traceback (most recent call last):
  File "/Users/hsouch/opt/miniconda3/lib/python3.9/site-packages/distributed/comm/tcp.py", line 211, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of 